<a href="https://colab.research.google.com/github/CodingTomo/PyTorch-Tutorials/blob/master/PyTorch_Ottimizzazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ottimizzazione

In questo notebook usiamo *autograd* di PyTorch trovare **punti critici** di funzioni. Nella pratica questo è fondamentale quando la funzione oggetto di studio è una qualche funzione di **costo** e le performance del modello dipendono da quanto si è bravi nel rintracciare il **minimo** di questa funzione.

In [0]:
import torch

Iniziamo cercando il punto critico della funzione $f(x)=x^2$. Lo faremo implementando la *discesa del gradiente*, cioè quell'algoritmo che trova minimi locali di una funzione seguendo la direzione opposta indicata dal gradiente. Il cuore dell'algoritmo è l'aggiornamento del punto su cui calcolare il gradiente successivo, ovvero: $$x_{t+1} = x_{t} - \lambda \nabla_x f (x_t),$$ dove $\lambda$ è un parametro che controlla la lunghezza del passo da $x_t$ a $x_{t+1}$ ed è detto *learning rate*.

**Osservazione**: ricordiamo che il gradiente di una funzione calcolato in un punto $x_0$ è un vettore ortogonale alla curva di livello della funzione passante per quel punto e la cui direzione mira al massimo locale della funzione. 

In [0]:
def f(x):
  return x ** 2

**Soluzione 1**: aggiornamento con variabile d'appoggio.

In [0]:
x_0=3.0
l_rate=0.1

x = torch.tensor([x_0], requires_grad=True)
y = f(x)

for i in range(5):
  y.backward()

  print('Il valore della derivata di f in {} è {}'.format(x.detach().numpy(),x.grad.detach().numpy()))
  print('Il valore atteso della derivata di f in {} è {}'.format(x.detach().numpy(), 2*x.detach().numpy() ))

  x_next = (x - l_rate * x.grad).detach()
  x = x_next.detach().requires_grad_(True)

  y = f(x)

  print('Il valore di f in {} è {}'.format(x.detach().numpy(),y.detach().numpy()))

  print( ' \n --- fine iterazione numero {} ----  \n '.format(i+1))

**Soluzione 2**: aggiornamento *inplace*.

In [0]:
x_0=3.0
l_rate=0.1

x = torch.tensor([x_0], requires_grad=True)
y = f(x)

for i in range(5):
  y.backward()

  print('Il valore della derivata di f in {} è {}'.format(x.detach().numpy(),x.grad.detach().numpy()))
  print('Il valore atteso della derivata di f in {} è {}'.format(x.detach().numpy(), 2*x.detach().numpy() ))

  with torch.no_grad():
    x -= l_rate * x.grad 
  
  #x = (x - l_rate * x.grad).detach().requires_grad_(True) # funziona al posto del contesto torch.no_grad

  x.grad.zero_()
  y = f(x)

  print('Il valore di f in {} è {}'.format(x.detach().numpy(),y.detach().numpy()))

  print( ' \n --- fine iterazione numero {} ----  \n '.format(i+1))

**Osservazione**: L'aggiornamento *inplace* di $x$ è particolarmente delicato. Vogliamo che il nuovo tensore non contenga la storia dell'aggiornamento e che il gradiente pre-esistente non si accumuli.  

**Esercizio**: Capire che differenza c'è tra le due soluzioni proposte. Variare il parametro *l_rate* e capire che impatti ha sull'algoritmo.

Nella pratica non è necessario implementare algoritmi come la discesa del gradiente poichè PyTorch implemeta la classe **optim** che contiene molte procedure di ottimizzazione fra cui quello appena visto.

In [0]:
import torch.optim as optim

parameters = [x] # in un caso reale contiene un iterabile con i parametri del modello da ottimizzare 

optimizer = optim.SGD(parameters, lr=0.01, momentum=0.9)
optimizer = optim.Adam(parameters, lr=0.01)
optimizer = optim.Adadelta(parameters, lr=0.01)
optimizer = optim.Adagrad(parameters, lr=0.01)
optimizer = optim.RMSprop(parameters, lr=0.01)
optimizer = optim.LBFGS(parameters, lr=0.01)

# ... elenco non esaustivo!

Utilizzando la stessa funzione $f$ degli esempi precedenti, sfuttiamo l'implementazione dell'algoritmo **SGD** di optim per cercare il minimo di $f$.

[Ulteriori informazioni sull'algoritmo SDG](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

In [0]:
x0 = 3
l_rate=0.1

x = torch.tensor([x_0], requires_grad=True)
y = f(x)

optimizer =  optim.SGD([x], lr=l_rate, momentum=0.3) # nel caso esempio passiamo la nostra unica variabile

for i in range(10):
    
  y.backward()
  optimizer.step()
  optimizer.zero_grad()
  y=f(x)

  print('Il valore di f in {} è {}'.format(x.detach().numpy(),y.detach().numpy()))

Spesso nella pratica può essere utile aggiustare il **leaning-rate** dinamicamente. Pytorch offre diverse funzionalità per controllare questo parametro in fase di addestramento.

Di seguito un elenco non esaustivo.
```
optim.lr_scheduler.LambdaLR
optim.lr_scheduler.ExponentialLR
optim.lr_scheduler.MultiStepLR
optim.lr_scheduler.StepLR

```



Utilizzando la stessa funzione $f$ degli esempi precedenti sfuttiamo l'implementazione dell'algoritmo **SGD** e dello scheduler **ExponentialLR**.

L'aggiornamento di questo scheduler segue la legge: $$\lambda_{t}=\lambda_{0}*\gamma^t,$$
dove $\lambda_{t}$ è il learning rate al passo $t$ e $\gamma$ un parametro nell'intervallo $(0,1)$ che controlla la velocità di decadimento.

In [0]:
x0 = 8.0
l_rate = 0.3

x = torch.tensor([x0]).requires_grad_()
optimizer = optim.SGD([x], lr=l_rate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.9)

for i in range(10):
  
    y.backward()
    optimizer.step()
    scheduler.step()

    optimizer.zero_grad()
    y = f(x)

    print('Il valore di f in {} è {} e learning rate attuale vale {}'
      .format(x.detach().numpy(),y.detach().numpy(),optimizer.param_groups[0]['lr']))

Un **esempio** completo.

In [0]:
from matplotlib import pyplot as plt
import torch.optim as optim
import torch
import numpy

In [0]:
def g(x):
    return x ** 2 / 20 + x.sin().tanh()

def plot(coord_x,coord_y):
   x_points = torch.linspace(-10, 10, 500)
   y_points = x_points ** 2 / 20 + x_points.sin().tanh()
   plt.scatter(coord_x, coord_y, color='red')
   plt.plot(x_points.numpy(), y_points.numpy())

In [0]:
x_0=8.0
l_rate=0.4

x=torch.tensor([x_0], requires_grad=True)
y=g(x)

coord_x=numpy.array([x_0])
coord_y=numpy.array([y.detach().numpy()])

optimizer = optim.Adam([x], lr=l_rate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.9)

for i in range(50):
  y.backward()
  optimizer.step()
  scheduler.step()
  optimizer.zero_grad()
  y=g(x)

  coord_x = numpy.append(coord_x, [x.detach().numpy()])
  coord_y = numpy.append(coord_y, [y.detach().numpy()])

plot(coord_x,coord_y)

**Esercizio**: ripetere l'esempio precedente usando una funzione a piacere definita da $\mathbb{R}^2$ in $\mathbb{R}$ e visualizzarne il grafico in tre dimensioni.